# Calculating mixed layer depth  
Determine for each grid point for each August the max MLD using density threshold of 0.03 kg/m3 from the 10 m depth value

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import warnings
import gcsfs
from xhistogram.xarray import histogram
import intake
import util
import gsw
import dask

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [3]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
col

pangeo-cmip6-ESM Collection with 28657 entries:
	> 10 activity_id(s)

	> 23 institution_id(s)

	> 48 source_id(s)

	> 29 experiment_id(s)

	> 86 member_id(s)

	> 23 table_id(s)

	> 190 variable_id(s)

	> 7 grid_label(s)

	> 28657 zstore(s)

	> 59 dcpp_init_year(s)

In [4]:
cat = col.search(experiment_id=['historical'], institution_id='NCAR', table_id='Omon', member_id='r1i1p1f1', variable_id=['thetao','so','siconc'], grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
14574,CMIP,NCAR,CESM2,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/NCAR/CESM2/historical/r1i1p1f1...,NaN
14580,CMIP,NCAR,CESM2,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/NCAR/CESM2/historical/r1i1p1f1...,NaN


In [5]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})
dset_dict.keys()

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)


dict_keys(['CMIP.NCAR.CESM2.historical.Omon.gn'])

In [6]:
for i in dset_dict:
    cthetao = xr.apply_ufunc(gsw.CT_from_pt, dset_dict[i].so, dset_dict[i].thetao, dask='parallelized',
                                             output_dtypes=[float,]).rename('cthetao').to_dataset() 
    dset_dict[i] = xr.merge([cthetao, dset_dict[i]])

for i in dset_dict:    
    pdens=xr.apply_ufunc(gsw.density.sigma0,dset_dict[i].so, dset_dict[i].cthetao, dask='parallelized', 
                        output_dtypes=[float, ]).rename('pdens').to_dataset()
    dset_dict[i] = xr.merge([pdens, dset_dict[i]])
    print(dset_dict[i])


<xarray.Dataset>
Dimensions:    (d2: 2, lev: 60, member_id: 1, nlat: 384, nlon: 320, time: 1980, vertices: 4)
Coordinates:
  * time       (time) float64 6.749e+05 6.749e+05 ... 7.351e+05 7.351e+05
  * lev        (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
  * nlat       (nlat) int32 1 2 3 4 5 6 7 8 ... 377 378 379 380 381 382 383 384
  * nlon       (nlon) int32 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: d2, vertices
Data variables:
    pdens      (member_id, time, lev, nlat, nlon) float64 dask.array<chunksize=(1, 8, 60, 384, 320), meta=np.ndarray>
    cthetao    (member_id, time, lev, nlat, nlon) float64 dask.array<chunksize=(1, 8, 60, 384, 320), meta=np.ndarray>
    lat_bnds   (nlat, nlon, vertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
    lat        (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon_bnds   (nlat, nlon, vertices) float3

In [7]:
# need to find index of depth or lev that is closest to 10m. If units are in centimeters, convert to meters

if 'depth' in dset_dict[i]:
    dset_dict[i].depth = xr.Dataset.rename({'depth':'lev'})
    print(dset_dict[i])
if dset_dict[i].lev.units == 'centimeters':
    dset_dict[i].lev.values = dset_dict[i].lev.values/100
    dset_dict[i].lev.attrs['units'] = 'm'
    print(dset_dict[i].lev)

# interpolate density data
for i in dset_dict:
    dsi = dset_dict[i].pdens.interp(lev=np.linspace(0,20,21))
    surf_dens = dsi.sel(lev = 10)
#     print(surf_dens)
#     surf_dens.isel(time=0).plot();


<xarray.DataArray 'lev' (lev: 60)>
array([5.000000e+00, 1.500000e+01, 2.500000e+01, 3.500000e+01, 4.500000e+01,
       5.500000e+01, 6.500000e+01, 7.500000e+01, 8.500000e+01, 9.500000e+01,
       1.050000e+02, 1.150000e+02, 1.250000e+02, 1.350000e+02, 1.450000e+02,
       1.550000e+02, 1.650984e+02, 1.754790e+02, 1.862913e+02, 1.976603e+02,
       2.097114e+02, 2.225783e+02, 2.364088e+02, 2.513702e+02, 2.676542e+02,
       2.854837e+02, 3.051192e+02, 3.268680e+02, 3.510935e+02, 3.782276e+02,
       4.087846e+02, 4.433777e+02, 4.827367e+02, 5.277280e+02, 5.793729e+02,
       6.388626e+02, 7.075633e+02, 7.870025e+02, 8.788252e+02, 9.847059e+02,
       1.106204e+03, 1.244567e+03, 1.400497e+03, 1.573946e+03, 1.764003e+03,
       1.968944e+03, 2.186457e+03, 2.413972e+03, 2.649001e+03, 2.889385e+03,
       3.133405e+03, 3.379793e+03, 3.627670e+03, 3.876452e+03, 4.125768e+03,
       4.375392e+03, 4.625190e+03, 4.875083e+03, 5.125028e+03, 5.375000e+03])
Coordinates:
  * lev      (lev) float64 

In [8]:
for i in dset_dict:
    dens_diff = dset_dict[i].pdens - surf_dens
#     print(dens_diff.isel(time=0).min().values)
    dens_diff = dens_diff.where(dens_diff > 0.03)
#     print(dens_diff.shape)
    min_dens_diff = dens_diff.lev.where(dens_diff==dens_diff.min(['lev'])).max(['lev'])

In [ ]:
min_dens_diff.plot()

In [ ]:
temp = min_dens_diff.isel(member_id=-1, time=slice(-13,-1)).sel(nlon=slice(200,250), nlat=slice(0,80)).mean(['nlon','nlat'])

In [ ]:
temp.load()

In [ ]:
temp.plot()

In [ ]:
dens_diff.isel(time=-1, member_id=-1).isel(nlat=100, nlon=100).min(['lev']).values

In [ ]:
dens_diff.min([])

In [ ]:
min_dens_diff.isel(time=-1, member_id=-1)[0,100,100]

In [ ]:
dens_diff.isel(member_id=0, time=-1, nlat=100, nlon=100).min(['lev']).values

In [ ]:
dset_dict[i].pdens.isel(time=0).max().values

In [ ]:
import cartopy.feature as cfeature
import numpy.ma as ma
fig = plt.figure(figsize=(8,4))
for n,i in enumerate(dset_dict):
    print(n)
    ax = plt.subplot(1,2,n+1,projection=ccrs.SouthPolarStereo())
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
    ax.set_extent([0.005, 360,  -90, -50], crs=ccrs.PlateCarree())
        

    mydata = dset_dict[i].pdens.isel(time=0,lev=0).values[0,:,:]
    print(mydata.shape)
    mydata = ma.masked_where(dset_dict[i].lat.values>0.,mydata)
    print(dset_dict[i].lat.values.shape)
    
    ax.pcolormesh(dset_dict[i].lat, dset_dict[i].lon, mydata,transform =ccrs.PlateCarree())
    #ax.coastlinesmydata()
plt.tight_layout()
plt.show(); plt.close()

In [ ]:
for n,i in enumerate(dset_dict):
    dset_dict[i].pdens.isel(time=0,lev=0).plot(); plt.show(); plt.close()